# Machine Learning/ Deep Learning + Internet of Things + Social Media of Things

# Send whatsapp, telegram message, SMS, mail and launch instance, volume by recognising your Face.


# Step 1 - Create Training Data

In [1]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 200: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-0b9a552b0eb8>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


# Step 2 - Train Model

In [2]:
cap.release()

In [3]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


# Step 3 - Run Our Facial Recognition

## How this program works :-

Case 1:-
    If model recognises my face then this program will send whatsapp message, telegram message, mail, SMS to me

Case 2:-
    If model recognises another face then program will launch the EC2 instance, EBS volume and attach EBS volume with EC2 instance

Case 3:-
    If there is no faces then it will print "No face found, looking for face"

In [7]:
# importing all the important libraries
import cv2
import numpy as np
import os
from boltiot import Email, Sms, Bolt
import json, time
import requests
from twilio.rest import Client

# using haarcascade frontal face algorithm to detect face
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# function to detect the face
def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


NAME = "Bhavesh Sonewale"


# function for sending SMS
def sms():
    SID = "#######################################"
    AUTH_TOKEN = "###################################"
    FROM_NUMBER = +18029#######
    TO_NUMBER = +9193597#####
    
    sms = Sms(SID, AUTH_TOKEN, TO_NUMBER, FROM_NUMBER)
    
    print("Making request to Twilio to send a SMS")
    response = sms.send_sms("Face detected, This is face of " + NAME)
    print("Response received from Twilio is: " + str(response))
    print("Status of SMS at Twilio is :" + str(response.status))

    
    
# function for sending Mail
def mail():
    MAILGUN_API_KEY =  "5537e535###########################"
    SANDBOX_URL= "sandboxd87c############################.mailgun.org" 
    SENDER_EMAIL = 'Face_Detector@' + SANDBOX_URL
    RECIPIENT_EMAIL = 'bhaveshssonewale@gmail.com'
    
    mailer = Email(MAILGUN_API_KEY, SANDBOX_URL, SENDER_EMAIL, RECIPIENT_EMAIL)
    
    print("Making request to Mailgun to send an email")
    response = mailer.send_email("Your Face detected", "This is face of " + NAME )
    response_text = json.loads(response.text)
    print("Response received from Mailgun is: " + str(response_text['message']))

    

# function for sending messages via Telegram
def telegram():
    telegram_bot_id = "bot1861484724:AAFsX#############################"
    telegram_chat_id = "@face_det###########"
    
    url = "https://api.telegram.org/" + telegram_bot_id + "/sendMessage"
    telegram_message = "Face Detected, this is face of " + NAME
    data = {
        "chat_id": telegram_chat_id,
        "text": telegram_message
    }
    
    response = requests.request(
        "POST",
        url,
        params=data
    )
    
    print("This is the Telegram URL")
    print(url)
    print("This is the Telegram response")
    print(response.text)
    telegram_data = json.loads(response.text)
    
    
    
def whatsapp():
    SID = "AC0ced938e3765b###########################"
    AUTH_TOKEN = "111bd36dc##########################"
    FROM_NUMBER = +180299#####
    TO_NUMBER = +9193597#####
    
    client = Client(SID, AUTH_TOKEN) 
    
    whatsapp_message = client.messages.create( 
        from_='whatsapp:+141552#####',  
        body='Face Detected, this is face of '+ NAME,
        to='whatsapp:'+ str(TO_NUMBER)
    )
    print(whatsapp_message.sid)
    print("message sent")
    
    
    
# function to launch EC2 instance, EBS volume and to attach EBS volume with EC2 instance
def aws():
    os.system("terraform init")
    os.system("terraform apply --auto-approve")
    print("Ec2 instance created\nVolume created\n 5 GB Volume attached to instance")
    
    
    
# function for buzzing the buzzer
def buzzer():
    API_KEY = '41b0303d-4e04-43b1-ac###################'
    DEVICE_ID = 'BOLT##########'
    
    mybolt = Bolt(API_KEY, DEVICE_ID)
    
    buzzer_response = mybolt.digitalWrite(0,'HIGH')
    time.sleep(5) # for buzzing the buzzer till 5 sec
    buzzer_response = mybolt.digitalWrite(0,'LOW')
    
    
    
# Open Webcam
cap = cv2.VideoCapture(0)



# 
while True:
    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        # harry_model.predict(face)
        
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
    
    
        if confidence > 90:
            cv2.putText(image, "Hey Bhavesh", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
            #os.system("chrome https://www.google.com/search?q=vimal+daga")
            #os.system("notepad")
            
            sms()
            mail()
            telegram()
            whatsapp()
           
            break
         
        else:
            cv2.putText(image, "I dont know, who are you ?", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            
            #os.system("cd C:/Users/Asus/Desktop/10-06-2021/Face Recognition")
            #os.system("notepad")
            
            aws()
            buzzer()
            
            break
    
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
    
    """         
    except Exception as e: 
        print ("Error occured: Below are the details")
        print (e)
    """
    
    if cv2.waitKey(10) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()

<>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-7-5b1bf2c5c161>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Ec2 instance created
Volume created
 5 GB Volume attached to instance
